# ---------------------------TRABALHO FINAL DE MMD ---------------------------
##                                                 Análise de Dados do Zapimóvel 
##                              - Preços de Apartamentos na Zona Sul do Rio de Janeiro -
###                                              Antônio Luís Sombra de Medeiros


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import pylab as pl
import pandas as pd
import os
from IPython.core.display import HTML

import matplotlib.mlab as mlab
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly.graph_objs import *
#from plotly.graph_objs import Scatter, Figure, Layout
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)

from IPython.display import Image
%matplotlib inline

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)

height has been deprecated.



In [4]:
x = pd.read_excel("zap_resultado1400_2017-05-29.xlsx")

# Os Dados

##### Dados do dia 29/05/2017 obtidos através de um Scrapper em R ( Código no arquivo "crawler_zapmoveis.R") do site www.zapimóveis.com.br, na categoria Venda - Zona Sul ---> Foram obtidos 35000 anúncios, distribuídos em 1400 páginas com 25 anúncios em cada.

#### Segue abaixo um extrato da tabela obtida

In [3]:
x.head(5)

,link,preco,bairro,endereco,quartos,area,suites,vagas,descricao
1,https://www.zapimoveis.com.br/lancamento/apart...,Sob consulta,Botafogo,Rua Real Grandeza •\r\n ...,2 a 3 quartos,66 a 97m2,1 suíte,1 a 2 vagas,Apartamento na Planta\r\n Sofis...
2,https://www.zapimoveis.com.br/lancamento/apart...,R$ 1.380.000,Botafogo,Rua Sorocaba •\r\n ...,2 a 3 quartos,93 a 110m2,1 suíte,1 a 2 vagas,Apartamento Pronto Para Morar\r\n ...
3,https://www.zapimoveis.com.br/lancamento/apart...,R$ 1.380.000,Botafogo,Rua Pinheiro Guimarães •\r\n ...,2 a 4 quartos,105 a 131m2,1 a 2 suítes,1 a 3 vagas,Apartamento Pronto Para Morar\r\n ...
4,https://www.zapimoveis.com.br/superdestaque/ve...,R$ 1.350.000,Copacabana,Rua Marechal Mascarenhas de Mo...,2 quartos,85m2,1 suíte,2 vagas,Apartamento\r\n Prédio excelent...
5,https://www.zapimoveis.com.br/superdestaque/ve...,R$ 3.500.000,Leblon,Rua General Venâncio Flores •\...,3 quartos,125m2,1 suíte,1 vaga,"Apartamento\r\n Alto padrão, to..."


In [5]:
# Vamos confirmar a quantidade de anúncios obtida, vendo o número valores diferentes de null em cada coluna
df = x.copy()
("Numero de apts que possuem valores não nulos pra cada coluna")
df.count()

link         35000
preco        35000
bairro       35000
endereco     35000
quartos      33600
area         35000
suites       14788
vagas        15368
descricao    35000
dtype: int64

# Limpeza dos Dados

In [6]:
#Substituo os NaN nas suites e vagas por "0"

df.suites = df.suites.fillna("0")
df.vagas = df.vagas.fillna("0")

#Tiro os apts que não tem quartos (podem ser apenas vagas sendo vendidas, em vez de apartamentos inteiros)

df = df.dropna(subset=["quartos"]) 

#Eliminar o "m2" do final do valor das areas

df['area'] = df['area'].map(lambda x: x.replace("m2",""))

#Eliminar o "vagas" ao final das vagas


df.vagas = df.vagas.str.rstrip("vagas")

# Tiro as entradas que tem preço "Sob consulta"

df = df[df.preco.str.contains("Sob consulta") == False]

# Tirar "R$" da frente do valor do preço e os pontos e transformar coluna para inteiro

df['preco'] = df['preco'].replace('[\R$]','',regex=True)
df['preco'] = df['preco'].replace('[\.]','',regex=True).astype(float)

#Vamos contar o número de apartamento por número de quartos para termos uma idéia do dataset
df.quartos.value_counts()

1 quarto         12840
2 quartos         9598
3 quartos         6828
4 quartos          707
2 a 3 quartos      600
2 a 4 quartos      589
5 quartos           28
Name: quartos, dtype: int64

In [7]:
#Tomar apenas o primeiro valor dos quartos, já que sempre o anunciado para mais de um apt é "A partir de X reais"
#Supõe-se que o preço do imóvel divulgado é para o valor mínimo de quartos anunciado. O mesmo pra suítes, vagas e área.

df["quartos"] = df["quartos"].map((lambda x: x[0])).astype(int)
df["suites"] = df["suites"].map((lambda x: x[0])).astype(int)
df["vagas"] = df["vagas"].map((lambda x: x[0])).astype(int)
df["area"] = df["area"].map((lambda x: x.split(' ', 1)[0])).astype(int)

# Tomar endereco apenas como o nome da rua
df["endereco"] = df["endereco"].map(lambda x: x.split('•', 1)[0].rstrip().lstrip()) # split() only once

#Defino uma nova coluna chamada "preço_m2", que será o preço/metro quadrado
df["preco_m2"] = df.preco/df.area

#O novo número de apartamentos por número de quartos será:
df.quartos.value_counts()

1    12840
2    10787
3     6828
4      707
5       28
Name: quartos, dtype: int64

In [8]:
df = df.reset_index()
df.to_excel("zap_resultado_limpo.xlsx")

In [9]:
data = pd.read_excel("zap_resultado_limpo.xlsx")

# Análise Geral

- Quais as principais diferenças entre os apartamentos de cada bairro da Zona Sul do Rio de Janeiro?
- Quais os bairros com os aluguéis mais caros?
- Quantos quartos os apartamentos têm em média?
- A oferta de apartamentos de um bairro influencia o preço médio do aluguel?

Vamos buscar responder essas e outras perguntas

## Valor de venda

Quais os bairros com maiores e menores valores de aluguel?
Qual o bairro com melhor média Preço/m2?


In [14]:
# Estatísticas do Preço
data["preco"].describe()


count       31,190.00
mean     1,407,323.44
std      1,668,828.58
min        360,000.00
25%        600,000.00
50%        710,000.00
75%      1,360,000.00
max     14,000,000.00
Name: preco, dtype: float64

In [16]:
data.head(5)

,index,link,preco,bairro,endereco,quartos,area,suites,vagas,descricao,preco_m2
0,2,https://www.zapimoveis.com.br/lancamento/apart...,1380000,Botafogo,Rua Sorocaba,2,93,1,1,Apartamento Pronto Para Morar\r\n ...,"14,838.71"
1,3,https://www.zapimoveis.com.br/lancamento/apart...,1380000,Botafogo,Rua Pinheiro Guimarães,2,105,1,1,Apartamento Pronto Para Morar\r\n ...,"13,142.86"
2,4,https://www.zapimoveis.com.br/superdestaque/ve...,1350000,Copacabana,Rua Marechal Mascarenhas de Morais,2,85,1,2,Apartamento\r\n Prédio excelent...,"15,882.35"
3,5,https://www.zapimoveis.com.br/superdestaque/ve...,3500000,Leblon,Rua General Venâncio Flores,3,125,1,1,"Apartamento\r\n Alto padrão, to...","28,000.00"
4,6,https://www.zapimoveis.com.br/superdestaque/ve...,2900000,Leblon,Rua Sambaíba,2,206,0,2,Apartamento\r\n Apartamento tri...,"14,077.67"


## Quantidade de ofertas x Preço

In [17]:
data.preco.iplot(xTitle='Preço em Reais (escala por 300.000.000 R$) ',\
                 yTitle='Quantidade de apartamentos', kind='histogram', color = "#e5450f")


Observa-se que há grandes outliers, que são apartamentos com ofertas acima de 8 Milhões de reais. Se tirarmos esses pouco número de ofertas outliers, teremos um novo gráfico, mais fácil de visualizar: 

In [18]:
data.preco[data.preco < 7560000].iplot(xTitle='Preço em Reais (escala por 300.000.000 R$) ',\
                                       yTitle='Quantidade de apartamentos', kind='histogram', color = "#e5450f")


## Quantidade de ofertas por No de quarto/suítes/vagas

In [20]:
df = data[['quartos',
       'suites', 'vagas']]
df.iplot(xTitle='Quantidade de quartos/suítes/vagas', yTitle='Quantidade de apartamentos', kind='histogram')


## Área do apartamento

In [22]:
df = data[['area']]
df.iplot(xTitle='Área do apartamento (m2)', yTitle='Quantidade de apartamentos', kind='histogram', color = "#e5dc0f")


#### Há pontos com áreas muito grandes que distorcem o gráfico, como os 11 apartamentos com mais de 477m2. Assim vamos verificar o maior valor de área ao qual 99% dos apartamentos estejam limitados.
#### Para isso utilizaremos a função centile, que nos retornará o que queremos.
#### Logo em seguida plotaremos o mesmo gráfico mas para valores até esse limite.

In [23]:
def centile(arr, percentiles=[50]):
  result = {}

  s = np.sort(arr)
  n = len(s)

  for percent in percentiles: 
    R = float(n + 1) * percent / 100
    r, f = int(R), R - int(R)

    result['{0}%'.format(percent)] = float(s[r - 1]) + f * (s[r] - s[r - 1])

  return result

In [24]:
centile(data.area, percentiles=[99] )

{'99%': 227.0}

In [27]:
df = data[['area']][data['area']<227]
df.iplot(xTitle='Área do apartamento (m2)', yTitle='Quantidade de apartamentos', kind='histogram', color="#e5dc0f")


## Análise do preço/m2 a medida que a área do apt aumenta 



In [28]:
preco_m2 = [Bar(x=data.area,
            y=data.preco_m2)]

py.iplot(preco_m2)

#### Verifica-se que o preço/m2 tem uma leve tendência a aumentar a medida que a área total do apartamento aumenta, mas a variância é bem grande. Provavelmente porque há muitos fatores que podem influenciar o preço além da área total (como número de vagas etc, que podem variar significativamente o preço do apartamento)

## QUARTOS

### Quantos quartos têm em média os apartamentos?
### Vamos plotar a distribuição dos apartamentos por quarto

In [29]:
quartos_stats = data.groupby('quartos')[['preco','area','suites','vagas', 'preco_m2']]

quartos = data.groupby("quartos").size()

total_quartos = quartos.sum()
percentual_quartos = quartos/total_quartos

quartos_stats.describe()



preco                                                                                                  area                                                    suites                                        vagas                                     preco_m2                                                                     
            count         mean          std          min          25%          50%          75%           max     count   mean   std    min    25%    50%    75%    max     count mean  std  min  25%  50%  75%  max     count mean  std  min  25%  50%  75%  max     count      mean      std       min       25%       50%       75%       max
quartos                                                                                                                                                                                                                                                                                                                                         
1       12,840.00   692,896.11   244,253.99   360,000.00   600,000.00   680,000.00   710,000.00  1,500,000.00 12,840.00  51.68  9.61  30.00  45.00  50.00  60.00 100.00 12,840.00 0.33 0.47 0.00 0.00 0.00 1.00 1.00 12,840.00 0.12 0.32 0.00 0.00 0.00 0.00 1.00 12,840.00 13,218.65 2,970.54  7,307.69 10,888.89 12,950.82 14,166.67 18,970.59
2       10,787.00   970,336.70   624,299.44   400,000.00   450,000.00   690,000.00 1,380,000.00  4,600,000.00 10,787.00  84.65 26.85  54.00  60.00  80.00  93.00 206.00 10,787.00 0.43 0.72 0.00 0.00 0.00 1.00 2.00 10,787.00 0.71 0.70 0.00 0.00 1.00 1.00 3.00 10,787.00 10,603.66 3,613.04  6,666.67  7,500.00  8,625.00 13,142.86 40,000.00
3        6,828.00 3,188,132.25 2,511,256.70   740,000.00 1,070,000.00 2,200,000.00 3,500,000.00 14,000,000.00  6,828.00 149.74 48.88  75.00 125.00 140.00 165.00 430.00  6,828.00 1.17 1.07 0.00 0.00 1.00 1.00 3.00  6,828.00 1.05 1.15 0.00 0.00 1.00 2.00 3.00  6,828.00 19,005.09 9,447.70  7,400.00  9,208.47 17,322.83 21,875.00 42,857.14
4          707.00 3,696,807.61 2,492,359.63 1,200,000.00 1,900,000.00 2,900,000.00 4,490,000.00 14,000,000.00    707.00 208.98 60.76 110.00 170.00 200.00 240.00 400.00    707.00 1.40 1.02 0.00 1.00 1.00 2.00 4.00    707.00 1.90 0.92 0.00 1.00 2.00 3.00 4.00    707.00 17,702.07 9,222.60  5,642.63  9,882.35 16,818.18 21,000.00 41,176.47
5           28.00 5,300,000.00 1,422,048.60 4,000,000.00 4,000,000.00 4,000,000.00 6,800,000.00  6,800,000.00     28.00 345.14 16.25 328.00 328.00 360.00 360.00 360.00     28.00 2.86 2.03 1.00 1.00 1.00 5.00 5.00     28.00 2.39 1.52 1.00 1.00 1.00 4.00 4.00     28.00 15,577.82 4,886.06 11,111.11 11,111.11 11,111.11 20,731.71 20,731.71

In [30]:
quartos.iplot(xTitle='Número de quartos do apartamento ',\
                 yTitle='Quantidade de apartamentos', kind= 'bar', color="#FFaC51")

/Users/Sombra/anaconda3/lib/python3.5/site-packages/cufflinks/plotlytools.py:156: FutureWarning:

pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp



### Percentual de apartamento por No de Quartos:

In [31]:
print("Percentual de apartamento por número de quartos: ", percentual_quartos)

Percentual de apartamento por número de quartos:  quartos
1   0.41
2   0.35
3   0.22
4   0.02
5   0.00
dtype: float64


#### Note que a maioria dos quartos dos apartamentos à venda na Zona Sul do Rio de Janeiro possuem 1 quarto (41.16%)

### Quantidade de apartamentos pelo preço e número de quartos
#### Preços ordenados do menor para o maior

In [32]:
data.groupby('quartos').preco.value_counts().sort_index()

quartos  preco   
1        360000      1400
         365000        14
         377000        16
         400000        17
         430000        10
         440000        17
         450000        21
         475000        18
         490000      1400
         525000        15
         528000        13
         600000      1400
         630000      1400
         635000        15
         650000        26
         680000      1400
         685000        21
         700000      1400
         710000      1400
         790000      1400
         795000        17
         1290000     1400
         1500000       20
2        400000      1400
         450000      1400
         575000        10
         590000      1400
         649000        19
         650000        38
         660000        13
         680000        29
         690000      1427
         695000        18
         700000        12
         720000        24
         730000        11
         750000        20
         760000     

## BAIRROS

#### Quantos Bairros diferentes temos em anúncios na Zona Sul do Rio de Janeiro? Quais? Quantos Logradouros?
#### Quais bairros possuem mais anúncios de apartamentos? Quais os que possuem preços mais altos de apartamentos à venda?

In [33]:
for cat in ['bairro','endereco']:
    print("Unique values for category '{0}': \b {1} ".format(cat, data[cat].unique()))
    print()

print("\nNúmero de bairros diferentes: ", len(data["bairro"].unique()))
print("\nNúmero de enderecos (ruas) diferentes: ", len(data["endereco"].unique()))

Unique values for category 'bairro':  ['Botafogo' 'Copacabana' 'Leblon' 'Gávea' 'Jardim Botânico' 'Glória'
 'Lagoa' 'Santa Teresa' 'Leme' 'Ipanema' 'Laranjeiras' 'Humaitá' 'Urca'
 'Flamengo' 'Catete'] 

Unique values for category 'endereco':  ['Rua Sorocaba' 'Rua Pinheiro Guimarães'
 'Rua Marechal Mascarenhas de Morais' 'Rua General Venâncio Flores'
 'Rua Sambaíba' 'Rua Siqueira Campos' 'Marques de São Vicente'
 'Rua Abreu Fialho' 'Avenida Ataulfo de Paiva'
 'Rua Ministro Viveiros de Castro' 'Rua Voluntários da Pátria'
 'Rua Benjamim Constant' 'R Santa Clara' 'Avenida Epitácio Pessoa'
 'Rua Décio Vilares' 'Rua Santa Clara' 'Avenida Princesa Isabel'
 'Rua Pinto Martins' 'Rua Prefeito João Felipe'
 'Avenida Nossa Senhora de Copacabana' 'Rua Gustavo Sampaio'
 'Rua Constante Ramos' 'Rua Redentor' 'Rua Assunção'
 'Rua Baronesa de Poconé' 'Rua Henrique de Novais' 'Avenida Delfim Moreira'
 'Rua da Matriz' 'Rua Alberto de Campos' 'Rua Almirante Saddock de Sá'
 'Avenida Atlântica' 'Rua Presid

### Número de bairros diferentes:  15
### Número de enderecos (ruas) diferentes:  160    

In [34]:
#Crio uma nova dataframe chamada bairros_stats onde coloco meu describe por bairro


bairros_stats = data.groupby('bairro')[['preco','area','suites','vagas','quartos', 'preco_m2']].describe()


bairros_stats  

#Ex de pesquisa na dataframe: 
#bairros_stats.loc['Botafogo']

preco                                                                                                  area                                                   suites                                        vagas                                      quartos                                     preco_m2                                                                     
                    count         mean          std          min          25%          50%          75%           max     count   mean   std   min    25%    50%    75%    max     count mean  std  min  25%  50%  75%  max     count mean  std  min  25%  50%  75%  max     count mean  std  min  25%  50%  75%  max     count      mean      std       min       25%       50%       75%       max
bairro                                                                                                                                                                                                                                                                                                                                                                                              
Botafogo         5,052.00   921,543.94   527,657.50   490,000.00   490,000.00   690,000.00 1,380,000.00  6,800,000.00  5,052.00  77.17 27.98 45.00  45.00  80.00  93.00 328.00  5,052.00 0.43 0.57 0.00 0.00 0.00 1.00 5.00  5,052.00 0.75 0.54 0.00 0.00 1.00 1.00 4.00  5,052.00 1.89 0.70 1.00 1.00 2.00 2.00 5.00  5,052.00 11,604.11 2,548.21  6,956.52  8,625.00 10,888.89 14,743.59 20,731.71
Catete              94.00   910,755.32   239,511.72   430,000.00   649,000.00   998,000.00 1,080,000.00  1,150,000.00     94.00  75.18 16.61 42.00  56.00  85.00  87.00  87.00     94.00 0.69 0.46 0.00 0.00 1.00 1.00 1.00     94.00 0.89 0.31 0.00 1.00 1.00 1.00 1.00     94.00 2.59 0.68 1.00 2.00 3.00 3.00 3.00     94.00 11,988.48 1,062.99 10,238.10 11,471.26 11,589.29 12,413.79 13,855.42
Copacabana      12,202.00   785,931.90   359,378.32   360,000.00   680,000.00   710,000.00   915,000.00  4,490,000.00 12,202.00  68.52 37.71 30.00  48.00  54.00  84.00 350.00 12,202.00 0.28 0.46 0.00 0.00 0.00 1.00 2.00 12,202.00 0.18 0.41 0.00 0.00 0.00 0.00 2.00 12,202.00 1.49 0.81 1.00 1.00 1.00 2.00 4.00 12,202.00 12,184.72 2,738.04  5,685.71  9,473.68 12,000.00 12,962.96 28,000.00
Flamengo           443.00 1,687,110.51 1,007,555.10   528,000.00   950,000.00 1,600,000.00 2,000,000.00  4,500,000.00    443.00 159.01 95.46 45.00  87.00 136.00 200.00 430.00    443.00 0.89 0.77 0.00 0.00 1.00 1.00 3.00    443.00 0.92 0.71 0.00 0.00 1.00 1.00 3.00    443.00 2.92 0.84 1.00 2.00 3.00 3.00 5.00    443.00 11,000.82 2,793.00  5,642.63  9,080.46 11,111.11 12,222.22 21,000.00
Glória           1,450.00   589,434.48    41,813.09   400,000.00   590,000.00   590,000.00   590,000.00    900,000.00  1,450.00  69.88  6.83 41.00  70.00  70.00  70.00 120.00  1,450.00 0.01 0.10 0.00 0.00 0.00 0.00 1.00  1,450.00 0.98 0.15 0.00 1.00 1.00 1.00 1.00  1,450.00 1.99 0.19 1.00 2.00 2.00 2.00 3.00  1,450.00  8,457.90   280.23  7,500.00  8,428.57  8,428.57  8,428.57 10,476.19
Gávea            1,477.00 2,192,640.49    64,642.56 1,790,000.00 2,200,000.00 2,200,000.00 2,200,000.00  2,500,000.00  1,477.00 141.38  5.44 95.00 142.00 142.00 142.00 155.00  1,477.00 1.96 0.20 1.00 2.00 2.00 2.00 2.00  1,477.00 1.99 0.11 1.00 2.00 2.00 2.00 2.00  1,477.00 2.06 0.35 2.00 2.00 2.00 2.00 4.00  1,477.00 15,520.51   386.07 14,285.71 15,492.96 15,492.96 15,492.96 18,842.11
Humaitá            178.00 1,158,483.15   437,452.11   720,000.00   880,000.00 1,050,000.00 1,350,000.00  2,430,000.00    178.00  84.53 24.14 64.00  70.00  79.00  86.00 160.00    178.00 0.69 0.62 0.00 0.00 1.00 1.00 2.00    178.00 1.02 0.44 0.00 1.00 1.00 1.00 2.00    178.00 2.37 0.48 2.00 2.00 2.00 3.00 3.00    178.00 13,516.31 1,990.13 10,229.89 12,209.30 13,291.14 15,333.33 16,463.41
Ipanema            712.00 3,274,702.25 2,864,701.23   650,000.00 1,550,000.00 2,749,000.00 3,850,000.00 14,000,000.00    712.00 14

### Mediana dos preços por Bairro

In [ ]:
sort_keys(data.groupby("bairro")["preco"].mean()

In [43]:
bairro_preco = [Bar(x=data.bairro,
            y=data.groupby("bairro")["preco"].median())]

py.iplot(bairro_preco, color = "#a30fe5")


### Quantidade de Apartamentos por bairro

In [44]:
data_gb_bairro = data.groupby(['bairro'])

qtd_bairro = data_gb_bairro.size()

qtd_bairro.iplot(xTitle='Bairro',\
                 yTitle='Número de apartamentos', kind= 'bar', color="#a30fe5")


### Aluguel Mediano por Bairro e Qnt de Quartos

In [45]:
data_gb_bairro_quarto = data.groupby(['bairro', 'quartos'])

precos_bairro_quartos = data_gb_bairro_quarto[['preco']]
precos_bairro_quartos_median = precos_bairro_quartos.median()
precos_bairro_quartos_median

preco
bairro          quartos         
Botafogo        1         490000
                2         690000
                3        1150000
                4        1850000
                5        6800000
Catete          1         430000
                2         649000
                3        1020000
Copacabana      1         680000
                2         915000
                3        1050000
                4        1870000
Flamengo        1         528000
                2         899000
                3        1650000
                4        2300000
                5        4000000
Glória          1         420000
                2         590000
                3         900000
Gávea           2        2200000
                4        2000000
Humaitá         2         920000
                3        1350000
Ipanema         1         795000
                2        1250000
                3        2550000
                4        4100000
Jardim Botânico 2         899000
                3        3500000
                4        6500000
Lagoa           2        1450000
                3        7500000
                4        4100000
Laranjeiras     1         475000
                2         900000
                3        1000000
Leblon          1        1290000
                2        1500000
                3        2200000
                4        5500000
Leme            1         630000
                2        1100000
                3        1830000
Santa Teresa    2         425000
Urca            2        1149000
                3        3000000

In [46]:
qtd_apts_por_bairro_e_quartos = data_gb_bairro_quarto.size()
qtd_apts_por_bairro_e_quartos

bairro           quartos
Botafogo         1          1433
                 2          2814
                 3           726
                 4            66
                 5            13
Catete           1            10
                 2            19
                 3            65
Copacabana       1          8481
                 2          1604
                 3          1929
                 4           188
Flamengo         1            13
                 2           120
                 3           213
                 4            82
                 5            15
Glória           1            34
                 2          1400
                 3            16
Gávea            2          1431
                 4            46
Humaitá          2           112
                 3            66
Ipanema          1            31
                 2           128
                 3           399
                 4           154
Jardim Botânico  2            56
                 3

In [47]:
area_bairro_quarto_median = datadata_gb_bairro_quarto[['area']].median()
area_bairro_quarto_median

NameError: name 'datadata_gb_bairro_quarto' is not defined

In [ ]:
colors = ['#35a46c', '#5d87ac', '#ffac51', '#e5dc0f', 'a30f5']
marker= dict(size= 14,
                    line= dict(width=1),
                    color= c[i],
                    opacity= 0.3
                   ),


In [238]:
data = pd.read_excel("zap_resultado_limpo.xlsx")
data1 = data.loc[data["quartos"]==1]
data1

,index,link,preco,bairro,endereco,quartos,area,suites,vagas,descricao,preco_m2
5,7,https://www.zapimoveis.com.br/oferta/venda+apa...,700000,Copacabana,Rua Siqueira Campos,1,54,0,0,Apartamento\r\n Excelente Quart...,"12,962.96"
8,10,https://www.zapimoveis.com.br/oferta/venda+apa...,1290000,Leblon,Avenida Ataulfo de Paiva,1,68,1,0,Apartamento\r\n Ataulfo de Paiv...,"18,970.59"
9,12,https://www.zapimoveis.com.br/oferta/venda+apa...,600000,Copacabana,Rua Ministro Viveiros de Castro,1,50,0,0,Apartamento\r\n Excelente apart...,"12,000.00"
10,13,https://www.zapimoveis.com.br/oferta/venda+apa...,490000,Botafogo,Rua Voluntários da Pátria,1,45,0,0,Apartamento\r\n Apto na Volunta...,"10,888.89"
12,15,https://www.zapimoveis.com.br/oferta/venda+apa...,680000,Copacabana,R Santa Clara,1,48,0,0,Apartamento\r\n Excelente prédi...,"14,166.67"
14,17,https://www.zapimoveis.com.br/oferta/venda+apa...,790000,Copacabana,Rua Décio Vilares,1,61,1,1,Apartamento\r\n Prédio localiza...,"12,950.82"
16,19,https://www.zapimoveis.com.br/oferta/venda+apa...,360000,Copacabana,Avenida Princesa Isabel,1,38,0,0,Apartamento\r\n $ $ C.R.S. Imóv...,"9,473.68"
19,22,https://www.zapimoveis.com.br/oferta/venda+apa...,710000,Copacabana,Avenida Nossa Senhora de Copacabana,1,41,1,0,Apartamento\r\n Apartamento ref...,"17,317.07"
20,23,https://www.zapimoveis.com.br/oferta/venda+apa...,630000,Leme,Rua Gustavo Sampaio,1,60,0,0,Apartamento\r\n Ótimo apartamen...,"10,500.00"
28,32,https://www.zapimoveis.com.br/oferta/venda+apa...,700000,Copacabana,Rua Siqueira Campos,1,54,0,0,Apartamento\r\n Excelente Quart...,"12,962.96"


In [1]:
colors = ['#35a46c', '#5d87ac', '#ffac51', '#e5dc0f', 'a30f5']

data1 = data.loc[data["quartos"]==1]
data2 = data.loc[data["quartos"]==2]
data3 = data.loc[data["quartos"]==3]
data4 = data.loc[data["quartos"]==4]
data5 = data.loc[data["quartos"]==5]

trace1 = go.Scatter(
    x = data1["bairro"],
    y = data1["preco"],
    name = str(data1["area"])+"m2",
    mode = 'markers',
    marker = dict(
        size = 10,
        color = colors[0],
        
    )
)

trace2 = go.Scatter(
    x = data2["bairro"],
    y = data2["preco"],
    name = str(data2["area"])+"m2",
    mode = 'markers',
    marker = dict(
        size = 10,
        color = colors[1],
        
    )
)

trace3 = go.Scatter(
    x = data3["bairro"],
    y = data3["preco"],
    name = str(data3["area"])+"m2",
    mode = 'markers',
    marker = dict(
        size = 10,
        color = colors[2],
        
    )
)

trace4 = go.Scatter(
    x = data4["bairro"],
    y = data4["preco"],
    name = str(data4["area"])+"m2",
    mode = 'markers',
    marker = dict(
        size = 10,
        color = colors[3],
        
    )
)


trace5 = go.Scatter(
    x = data5["bairro"],
    y = data5["preco"],
    name = str(data5["area"])+"m2",
    mode = 'markers',
    marker = dict(
        size = 10,
        color = colors[4],
        
    )
)

data = [trace1, trace2, trace3, trace4, trace5]

layout = dict(title = 'Styled Scatter',
              yaxis = dict(zeroline = False),
              xaxis = dict(zeroline = False)
             )

fig = dict(data=data, layout=layout)
#py.iplot(fig, filename='styled-scatter')


NameError: name 'data' is not defined

In [158]:
data_gb_bairro_quarto['quartos'].count()

bairro           quartos
Botafogo         1          1433
                 2          2814
                 3           726
                 4            66
                 5            13
Catete           1            10
                 2            19
                 3            65
Copacabana       1          8481
                 2          1604
                 3          1929
                 4           188
Flamengo         1            13
                 2           120
                 3           213
                 4            82
                 5            15
Glória           1            34
                 2          1400
                 3            16
Gávea            2          1431
                 4            46
Humaitá          2           112
                 3            66
Ipanema          1            31
                 2           128
                 3           399
                 4           154
Jardim Botânico  2            56
                 3

In [156]:
data_gb_bairro_quarto = data.groupby(['bairro','quartos'])


In [ ]:
data2["price"][data2["bairro"] == ]

In [143]:
data2 = data.copy()
#data2["qtd_apt_bairro_quartos"] = qtd_apt_bairro_quartos

In [140]:
bairro_quartos.size()

bairro           quartos
Botafogo         1          1433
                 2          2814
                 3           726
                 4            66
                 5            13
Catete           1            10
                 2            19
                 3            65
Copacabana       1          8481
                 2          1604
                 3          1929
                 4           188
Flamengo         1            13
                 2           120
                 3           213
                 4            82
                 5            15
Glória           1            34
                 2          1400
                 3            16
Gávea            2          1431
                 4            46
Humaitá          2           112
                 3            66
Ipanema          1            31
                 2           128
                 3           399
                 4           154
Jardim Botânico  2            56
                 3

In [ ]:
bairro_quartos_preco_medio = data.groupby(['bairro','quartos'])[['preco','area','suites','vagas', 'preco_m2']]


In [134]:
bairro_quartos = data.groupby(['bairro','quartos'])[['preco','area','suites','vagas', 'preco_m2']]

bairros_stats = bairro_quartos.describe()

bairros_stats

#bairros_quartos = bairros_stats.groupby('quartos')

preco                                                                                        area                                                            suites                                                vagas                                                preco_m2                                                                                                  
                          count          mean           std        min        25%        50%        75%         max   count        mean        std    min    25%    50%    75%    max   count      mean       std  min  25%  50%  75%  max   count      mean       std  min   25%  50%   75%  max    count          mean           std           min           25%           50%           75%           max
bairro          quartos                                                                                                                                                                                                                                                                                                                                                                                     
Botafogo        1        1433.0  4.941975e+05  2.746858e+04   490000.0   490000.0   490000.0   490000.0    685000.0  1433.0   45.385206   2.594494   45.0   45.0   45.0   45.0   65.0  1433.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0  1433.0  0.023029  0.150047  0.0  0.00  0.0  0.00  1.0   1433.0  10889.768214  7.825052e+01  10538.461538  10888.888889  10888.888889  10888.888889  11607.142857
                2        2814.0  1.002431e+06  3.368575e+05   650000.0   690000.0   690000.0  1380000.0   1380000.0  2814.0   88.244136  10.514785   54.0   80.0   80.0   93.0  105.0  2814.0  0.464108  0.498799  0.0  0.0  0.0  1.0  1.0  2814.0  0.977612  0.184340  0.0  1.00  1.0  1.00  2.0   2814.0  11125.214681  2.730782e+03   8111.111111   8625.000000   8625.000000  13142.857143  18400.000000
                3         726.0  1.193588e+06  2.259067e+05   800000.0  1150000.0  1150000.0  1150000.0   2700000.0   726.0   83.119835  12.593078   78.0   78.0   78.0   78.0  140.0   726.0  0.983471  0.392695  0.0  1.0  1.0  1.0  3.0   726.0  1.103306  0.304568  1.0  1.00  1.0  1.00  2.0    726.0  14397.569215  1.385108e+03   8888.888889  14743.589744  14743.589744  14743.589744  19285.714286
                4          66.0  2.601061e+06  1.107924e+06  1600000.0  1662500.0  1850000.0  3150000.0   4490000.0    66.0  180.136364  52.937666  110.0  110.0  176.0  230.0  233.0    66.0  1.424242  0.498015  1.0  1.0  1.0  2.0  2.0    66.0  1.939394  0.676616  1.0  1.25  2.0  2.00  3.0     66.0  15006.418459  4.855290e+03   6956.521739   9421.936759  16818.181818  17897.727273  19270.386266
                5          13.0  6.800000e+06  0.000000e+00  6800000.0  6800000.0  6800000.0  6800000.0   6800000.0    13.0  328.000000   0.000000  328.0  328.0  328.0  328.0  328.0    13.0  5.000000  0.000000  5.0  5.0  5.0  5.0  5.0    13.0  4.000000  0.000000  4.0  4.00  4.0  4.00  4.0     13.0  20731.707317  3.786528e-12  20731.707317  20731.707317  20731.707317  20731.707317  20731.707317
Catete          1          10.0  4.300000e+05  0.000000e+00   430000.0   430000.0   430000.0   430000.0    430000.0    10.0   42.000000   0.000000   42.0   42.0   42.0   42.0   42.0    10.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0    10.0  0.000000  0.000000  0.0  0.00  0.0  0.00  0.0     10.0  10238.095238  1.917383e-12  10238.095238  10238.095238  10238.095238  10238.095238  10238.095238
                2          19.0  6.490000e+05  0.000000e+00   649000.0   649000.0   649000.0   649000.0    649000.0    19.0   56.000000   0.000000   56.0   56.0   56.0   56.0   56.0    19.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0    19.0  1.000000  0.000000  1.0  1.00  1.0  1.00  1.0     19.0  11589.285714  3.737668e-12  11589.285714  11589.285714  11589.285714  11589.285714  11589.285714
                3          65.0  1.061231e+06  6.252269e

### Para cada Bairro, achemos o Preço e o Número de Apts com aquele preço 

In [101]:
data.groupby('bairro').preco.value_counts()

bairro           preco   
Botafogo         690000      1409
                 490000      1400
                 1380000     1206
                 1150000      617
                 650000        31
                 800000        28
                 970000        27
                 1000000       26
                 685000        21
                 1850000       21
                 750000        20
                 1450000       18
                 1300000       17
                 1600000       17
                 1250000       16
                 1400000       16
                 1655000       15
                 3150000       15
                 1080000       14
                 1190000       14
                 1360000       14
                 660000        13
                 4490000       13
                 6800000       13
                 2700000       12
                 730000        11
                 900000        11
                 760000        10
                 95000

### Análise de Preço/Número de ofertas para cada tipo de anúncio --POR No DE SUÍTES--

In [ ]:
suites_stats = data.groupby('suites')[['preco','area','vagas','quartos', 'preco_m2']].describe()


suites_stats

In [ ]:
print_full(data.groupby('suites').preco.value_counts())

### Análise de Preço/Número de ofertas para cada tipo de anúncio --POR No DE VAGAS--

In [130]:
vagas_stats = data.groupby('vagas')[['preco','area','suites','quartos', 'preco_m2']].describe()
vagas_stats

preco                                                                                         area                                                             suites                                               quartos                                              preco_m2                                                                                                 
         count          mean           std        min        25%        50%        75%         max    count        mean        std    min    25%    50%    75%    max    count      mean       std  min  25%  50%  75%  max    count      mean       std  min  25%  50%  75%  max    count          mean          std           min           25%           50%           75%           max
vagas                                                                                                                                                                                                                                                                                                                                                                                      
0      19035.0  9.077925e+05  7.792121e+05   360000.0   490000.0   680000.0   915000.0   3500000.0  19035.0   69.320830  34.562536   30.0   48.0   60.0   70.0  319.0  19035.0  0.234568  0.430871  0.0  0.0  0.0  0.0  3.0  19035.0  1.561860  0.752702  1.0  1.0  1.0  2.0  4.0  19035.0  12404.748782  4378.285226   5642.633229   9473.684211  10892.857143  14166.666667  28000.000000
1       8321.0  1.099995e+06  6.222604e+05   525000.0   690000.0   790000.0  1380000.0   4500000.0   8321.0   89.892200  38.678620   30.0   70.0   80.0   94.0  430.0   8321.0  0.608461  0.560773  0.0  0.0  1.0  1.0  3.0   8321.0  2.124024  0.727880  1.0  2.0  2.0  3.0  5.0   8321.0  12072.254989  3684.752322   5685.714286   8625.000000  12950.819672  14743.589744  40000.000000
2       2172.0  2.478251e+06  1.456233e+06   865000.0  2200000.0  2200000.0  2200000.0  14000000.0   2172.0  147.445672  39.040428   75.0  142.0  142.0  142.0  400.0   2172.0  1.709945  0.581167  0.0  1.0  2.0  2.0  4.0   2172.0  2.428177  0.690728  2.0  2.0  2.0  3.0  4.0   2172.0  16501.526626  4824.465447   7100.000000  15492.957746  15492.957746  15492.957746  42857.142857
3       1630.0  7.280288e+06  1.203529e+06  3300000.0  7500000.0  7500000.0  7500000.0  14000000.0   1630.0  220.684049  15.348825  170.0  220.0  220.0  220.0  340.0   1630.0  2.844785  0.505179  1.0  3.0  3.0  3.0  4.0   1630.0  3.096319  0.339608  2.0  3.0  3.0  3.0  4.0   1630.0  32912.209755  4265.078902  17083.333333  34090.909091  34090.909091  34090.909091  41176.470588
4         32.0  6.621875e+06  1.496973e+05  6500000.0  6500000.0  6500000.0  6800000.0   6800000.0     32.0  282.281250  38.422301  251.0  251.0  251.0  328.0  328.0     32.0  4.406250  0.498991  4.0  4.0  4.0  5.0  5.0     32.0  4.406250  0.498991  4.0  4.0  4.0  5.0  5.0     32.0  23798.252113  2577.141896  20731.707317  20731.707317  25896.414343  25896.414343  25896.414343